In [1]:
from jLM.RegionBuilder import RegionBuilder
import jLM
import numpy as np
import matplotlib.pyplot as plt
import pickle, lzma
%load_ext Cython

fully occupied check

In [2]:
def fully_occupied_check(regions):
    # Create a combined region that is the logical OR of all input regions
    combined_region = np.logical_or.reduce(regions)
    
    # Create a binary matrix where unoccupied voxels are True
    unoccupied_voxels = ~combined_region
    
    # Count the number of unoccupied voxels
    num_unoccupied = np.sum(unoccupied_voxels)
    
    if num_unoccupied > 0:
        print(f"Warning: There are {num_unoccupied} unoccupied voxels in the lattice.")
    else:
        print("All regions fully occupy the lattice without any unoccupied voxels.")
    
    # Return the binary matrix of unoccupied voxels
    return unoccupied_voxels

region overlapping check function

In [3]:
def check_overlaps(regions, region_names):
    overlaps = []
    for i in range(len(regions)):
        for j in range(i+1, len(regions)):
            overlap = np.logical_and(regions[i], regions[j])
            overlap_count = np.sum(overlap)
            if overlap_count > 0:
                overlaps.append((region_names[i], region_names[j], overlap_count))
    return overlaps

# # List of regions to check
# regions_to_check = [
#     fitVacuoleCropRegion, fitMitoCropRegion, fitNucEnvCropRegion, 
#     fitMemCropRegion, fitNucPoresCropRegion, fit_pmERReg, fit_cecERReg, 
#     fit_tubERReg, fit_pmERRiboReg, fit_cecERRiboReg, fit_tubERRiboReg, 
#     fitCytoRiboReg, fitCytoReg, fitNucleoPlasmReg
# ]

# region_names = [
#     'vacuole', 'mitochondria', 'nuclearEnvelope', 'plasmaMembrane', 
#     'nuclearPores', 'pmaER', 'cecER', 'tubER', 'pmaRibosomes', 
#     'cecRibosomes', 'tubRibosomes', 'cytoRibosomes', 'cytoplasm', 'nucleoplasm'
# ]
# overlaps = check_overlaps(regions_to_check, region_names)

# if overlaps:
#     print("Overlaps detected:")
#     for overlap in overlaps:
#         print(f"Overlap between {overlap[0]} and {overlap[1]}: {overlap[2]} voxels")
# else:
#     print("No overlaps detected. Proceeding with lattice assignment.")

 ## 1.Volume Constraints for different ER regions

sheets are defined by the loops with 4 nodes/ 6 nodes. and vertically opposite angle < certain degree, e.g. $60\degree$

volume based on the whats left after the volume we have now.
$$sheetVol = 2.2 \%- tubuleVol$$

and from 2011 paper in WT(371), relatively percentage:

tub = 0.3

cec = 0.2

pma = 0.4
>West, Matt, Nesia Zurek, Andreas Hoenger, and Gia K. Voeltz. 2011. “A 3D Analysis of Yeast ER Structure Reveals How ER Domains Are Organized by Membrane Curvature.” Journal of Cell Biology 193 (2): 333–46. https://doi.org/10.1083/jcb.201011039.


In [4]:
latticeData = pickle.load(lzma.open("/data2/2024_Yeast_cryoET/ribsome_dis_modification/yeast-lattice.2.pkl.xz", "rb"))

siteMap = {n:i for i,n in enumerate(latticeData['names'])}
def boolLattice(x):
    return latticeData['lattice'] == siteMap[x]

# fitExtracellular = boolLattice("extracellular") 
# fitCellWall = boolLattice("cellWall")
# fitNucEnvCropRegion  = boolLattice("nuclearEnvelope") 
# fitNucEnvCropRegion = boolLattice("nucleoplasm") 

# fitMitoCropRegion = boolLattice("mitochondria") 
# fitVacuoleCropRegion = boolLattice('vacuole')

# fitMemCropRegion = boolLattice("plasmaMembrane")

# fitNucPoresCropRegion = boolLattice("nuclearPores")
# fitcytoplasmaCrop = boolLattice("cytoplasm") | boolLattice("ribosomes")
'''We use Env and Mem from the np load,
because the geometry is not in agreement with the radii, angles and centers provided in yeast-slides.ipynb
and it has only minor angle difference, so we use the np load to replace it.
'''
# fitVacuoleCropRegion = np.load("./npys/vacuole.npy")
# fitMitoCropRegion = np.load("./npys/mitochondria.npy")
fitCellWall = np.load("./npys/cellWall.npy")
fitNucEnvCropRegion = np.load("./npys/nucleoEnvelop.npy")
fitNucPoresCropRegion = np.load("./npys/nucleoPores.npy")
fitMemCropRegion = np.load("./npys/plasmMembrane.npy")

regenerate the cytoplasm region

In [5]:
fitcytoplasmaCrop = np.load("./npys/cytoPlasm.npy").astype(bool)
# Combine all three regions
raw_cytoplasm = RegionBuilder.convexHull(fitcytoplasmaCrop)
fitcytoplasmaCrop = raw_cytoplasm & ~ ( fitNucEnvCropRegion | fitNucPoresCropRegion | fitMemCropRegion)
# create extracellular region
fitExtracellular = ~ (fitNucEnvCropRegion | fitNucPoresCropRegion | fitcytoplasmaCrop | fitMemCropRegion|fitCellWall)

In [ ]:
#regions inside the cell membrane
total_voxels = np.sum(fitNucEnvCropRegion | fitNucPoresCropRegion | fitcytoplasmaCrop)
print("total voxels:",total_voxels)
# generate each part voxel number
voxel_er = total_voxels * 0.022
voxel_cecER = np.round(voxel_er * 0.2)
voxel_tubER = np.round(voxel_er * 0.3)
voxel_pmER  = np.round(voxel_er * 0.5)
print("voxel_cecER:",voxel_cecER, "voxel_tubER:",voxel_tubER, "voxel_pmER:",voxel_pmER)

In [7]:
lattice_spacing = 28.67 # nm

In [ ]:
# check the region overlap before generating the ER
regions_to_check = [fitNucEnvCropRegion, fitNucPoresCropRegion, fitMemCropRegion, fitcytoplasmaCrop, fitCellWall, fitExtracellular]
region_names = ['nuclearEnvelope', 'nuclearPores', 'plasmaMembrane', 'cytoplasm', 'cellWall', 'extracellular']
overlaps = check_overlaps(regions_to_check, region_names)

if overlaps:
    print("Overlaps detected:")
    for overlap in overlaps:
        print(f"Overlap between {overlap[0]} and {overlap[1]}: {overlap[2]} voxels")
else:
    print("No overlaps detected. Proceeding with lattice assignment.")
    
    
unoccupied_reg = fully_occupied_check(regions_to_check)

## 1.1 pmER and cecER
first generate these two parts because we need contact sites on them

Parameters to build Plasma Membrane and nucleus envelope from `yeast-lattice-slides_rerun.ipynb`:
```bash
# memParamsTx:    [60.98760268 91.85676356 60.98760269  3.39485089 -1.58976337 -0.90393655
#  96.         96.         96.        ]

nucParamsTx:    [ 29.68455031  36.50814389  29.6845503    0.16526633  -1.74251698
   0.56589521 131.86756778  80.78099102 112.38567174]
```

acquire the [radius, angles, centers] for the elipsoid already created

In [9]:
import copy
memParamsTx = [60.98760268, 91.85676356, 60.98760269,  3.39485089, -1.58976337, -0.90393655, 96.,         96.,         96.        ]
nucParamsTx = [ 29.68455031,  36.50814389,  29.6845503,    0.16526633,  -1.74251698, 0.56589521, 131.86756778,  80.78099102, 112.38567174]
cecERParamsTx = np.array(copy.deepcopy(nucParamsTx))
pmERParamsTx = np.array(copy.deepcopy(memParamsTx))
thick_cec = 36.0 #nm
thick_pm = 35.6 #nm
gap_pm_pm = 33.0 #nm
gap_cec_nuc = gap_pm_pm  # keep the same as pm 
# modify the radius to create sheets
cecERParamsTx[0:3] += (gap_cec_nuc + thick_cec) /lattice_spacing
pmERParamsTx[0:3] -= (gap_pm_pm + thick_pm) /lattice_spacing

In [10]:
def ellipseFn(builder, params, thickness=3):
    radius = params[:3]
    angles = params[3:6]
    center = params[6:9]
    dr = thickness/2
    radius = np.maximum(np.zeros(3), radius - dr)
    if thickness == 0:
        e = builder.ellipsoid(radius=params[:3], 
                              angles=angles, center=center)
        return e & ~builder.erode(e,1)
    else:
        return ( builder.ellipsoid(radius=radius+dr, 
                                   angles=angles, center=center)
                  & ~builder.ellipsoid(radius=radius-dr, 
                                       angles=angles, center=center))

fit a ellipsoid inside PM and outside the nuclues envelope(NE)

In [11]:
n = 192
cec_er = RegionBuilder(dims=(n,n,n))
fitcecERFullReg=ellipseFn(cec_er, cecERParamsTx , thickness= int(thick_cec/lattice_spacing))
pm_er = RegionBuilder(dims=(n,n,n))
fitpmERFullReg=ellipseFn(pm_er, pmERParamsTx , thickness= int(thick_pm/lattice_spacing))

cut to fit the volume constraints

In [ ]:
import numpy as np
import random
from collections import deque


# def randomly_delete_part(ellipsoid_shell, target_voxels=1000):
#     """
#     Randomly delete part of the ellipsoid shell while attempting to preserve connectivity.
    
#     Args:
#     - ellipsoid_shell: 3D numpy array representing the ellipsoid shell.
#     - target_voxel: targeted number of voxels to be preserved.
    
#     Returns:
#     - Modified ellipsoid shell with parts randomly deleted.
#     """
#     modified_shell = ellipsoid_shell.copy()
#     x_size, y_size, z_size = modified_shell.shape
#     # randomly find a voxel to start
#     queue = deque()
#     neighbors_26 = [
#     (-1, -1, -1), (0, -1, -1), (1, -1, -1),
#     (-1, 0, -1), (0, 0, -1), (1, 0, -1),
#     (-1, 1, -1), (0, 1, -1), (1, 1, -1),
#     (-1, -1, 0), (0, -1, 0), (1, -1, 0),
#     (-1, 0, 0), (1, 0, 0),
#     (-1, 1, 0), (0, 1, 0), (1, 1, 0),
#     (-1, -1, 1), (0, -1, 1), (1, -1, 1),
#     (-1, 0, 1), (0, 0, 1), (1, 0, 1),
#     (-1, 1, 1), (0, 1, 1), (1, 1, 1)
#     ]
#     while True:
#         start = (random.randint(0, x_size-1), random.randint(0, y_size-1), random.randint(0, z_size-1))
#         if modified_shell[start] == 1:
#             queue.append(start)
#             break
#     current_voxels = np.sum(ellipsoid_shell)
    
#     while current_voxels > target_voxels and len(queue) > 0:
#         # print("current voxels: ", current_voxels)
#         x, y, z = queue.popleft()
        

#         #delete it
#         if modified_shell[x, y, z] != 0:
#             modified_shell[x, y, z] = 0
#             current_voxels -= 1
        
#             # add neighbors to the queue
#             neighbors = [(x + dx, y + dy, z + dz) for dx, dy, dz in neighbors_26]
            
#             # Add neighboring voxels to the queue
#             for nx, ny, nz in neighbors:
#                 # boundary check 
#                 if modified_shell[nx, ny, nz] != 0 and 0 <= nx < modified_shell.shape[0] and 0 <= ny < modified_shell.shape[1] and 0 <= nz < modified_shell.shape[2] :
#                     queue.append([nx, ny, nz])
#                 # print("queue len:", len(queue))
#     return modified_shell

def randomly_delete_part(ellipsoid_shell, target_voxels=1000):
    """
    Randomly generate several regions in the ellipsoid shell to reach the target_voxels,
    with each region having a random size.
    
    Args:
    - ellipsoid_shell: 3D numpy array representing the ellipsoid shell.
    - target_voxels: targeted number of voxels to be preserved.
    
    Returns:
    - Modified ellipsoid shell with several preserved regions of varying sizes.
    """
    neighbors_26 = [
    (-1, -1, -1), (0, -1, -1), (1, -1, -1),
    (-1, 0, -1), (0, 0, -1), (1, 0, -1),
    (-1, 1, -1), (0, 1, -1), (1, 1, -1),
    (-1, -1, 0), (0, -1, 0), (1, -1, 0),
    (-1, 0, 0), (1, 0, 0),
    (-1, 1, 0), (0, 1, 0), (1, 1, 0),
    (-1, -1, 1), (0, -1, 1), (1, -1, 1),
    (-1, 0, 1), (0, 0, 1), (1, 0, 1),
    (-1, 1, 1), (0, 1, 1), (1, 1, 1)
    ]
    modified_shell = np.zeros_like(ellipsoid_shell, dtype=np.uint8)
    x_size, y_size, z_size = ellipsoid_shell.shape
    
    # Determine the number of regions (2 to 10)
    num_regions = np.random.randint(2, 11)
    print("num_regions:",num_regions)
    # Generate random weights for each region
    weights = np.random.uniform(0.2, 0.8, num_regions)
    weights /= weights.sum()  # Normalize weights
    
    total_preserved_voxels = 0
    
    for i in range(num_regions):
        if total_preserved_voxels >= target_voxels:
            break
        
        # Calculate target voxels for this region
        region_target = int(weights[i] * target_voxels)
        region_target = min(region_target, target_voxels - total_preserved_voxels)
        print("region for region {}: {}".format(i,region_target))
        # Randomly find a starting voxel
        attempts = 0
        while attempts < 100:  # Limit attempts to avoid infinite loop
            valid_indices = np.argwhere(ellipsoid_shell == 1)
    
            # Randomly choose one of these indices
            random_index = np.random.randint(0, len(valid_indices))
            start = np.array(valid_indices[random_index])
            if modified_shell[tuple(start)] == 0:
                break
            attempts += 1
        
        if attempts == 100:
            continue  # Skip this region if we can't find a valid starting point
        
        queue = deque([start])
        region_voxels = 0
        
        while queue and region_voxels < region_target and total_preserved_voxels < target_voxels:
            x, y, z = queue.popleft()
            
            if ellipsoid_shell[x, y, z] == 1 and modified_shell[x, y, z] == 0:
                modified_shell[x, y, z] = i + 1
                region_voxels += 1
                total_preserved_voxels += 1
                
                # Add neighbors to the queue
                for dx, dy, dz in neighbors_26:
                    nx, ny, nz = x + dx, y + dy, z + dz
                    if 0 <= nx < x_size and 0 <= ny < y_size and 0 <= nz < z_size:
                        if ellipsoid_shell[nx, ny, nz] == 1 and modified_shell[nx, ny, nz] == 0:
                            queue.append((nx, ny, nz))
                            
                
    
    
    return modified_shell

# Randomly delete part of the ellipsoid shell

cutcecERFullReg = randomly_delete_part(fitcecERFullReg,target_voxels=voxel_cecER)
cutpmERFullReg = randomly_delete_part(fitpmERFullReg,target_voxels=voxel_pmER)
# Convert regions to binary matrices
cutpmERFullReg_binary = (cutpmERFullReg > 0).astype(bool)
cutcecERFullReg_binary = (cutcecERFullReg > 0).astype(bool)

# Perform the operation
fitcytoplasmaCrop &= ~(cutpmERFullReg_binary | cutcecERFullReg_binary)

In [ ]:
regions_to_check.extend([cutpmERFullReg, cutcecERFullReg])
region_names.extend(['PM ER', 'CEC ER'])
overlaps = check_overlaps(regions_to_check, region_names)

if overlaps:
    print("Overlaps detected:")
    for overlap in overlaps:
        print(f"Overlap between {overlap[0]} and {overlap[1]}: {overlap[2]} voxels")
else:
    print("No overlaps detected. Proceeding with lattice assignment.")
    
    
unoccupied_reg = fully_occupied_check(regions_to_check)

## 1.2 Rebuild the Tubular ER

In [14]:
def Diagonal_Line_3d(*,matrix, start, end, value, change=True, fitcytoplasmaCrop=fitcytoplasmaCrop, force=False,max_overlap=10):
    """Draw a line in matrix from start to end to check if overlapped or not.`
       Parameters:
            matrix: where to add line
            start, end: start and end point
            value: which value to add
            change: True= add line back to matrix, False= just do overlapping test 
            add function to check not overlap with other organelles and nucleus 
            fitcytoplasmaCrop: areas not to be overlapped
            force: if True, print out force  of failure, and force draw line
            max_overlap: tolerance of max overlap
       Return: 
            modified:   True= modified or able to draw the line , 
                        False= original or not able to draw the line
    """
    temp =  np.zeros_like(matrix)
    modified = False
    # A simple linear interpolation between the two points
    num_points = max(abs(end - start)) * 2  # Number of points in line
    # first check if the 
    for point in zip(np.linspace(start[0], end[0], num_points, dtype=int),
                     np.linspace(start[1], end[1], num_points, dtype=int),
                     np.linspace(start[2], end[2], num_points, dtype=int)):
        temp[tuple(point)] = value
    # print("overlap", np.sum(temp & ~ fitcytoplasmaCrop))
    unoccpied_region = fitcytoplasmaCrop & ~ (cutpmERFullReg | cutcecERFullReg)
    if np.sum(temp & ~ unoccpied_region) <= max_overlap:
        # tolerance is 2, with 2 contact sites 
        # not  overlapped, start and end point need to be considered
        modified = True
        if change:
            # combine temp into matrix
            matrix[temp > 0] = value
    else:
        if force:
            
            print("force draw line")
            # force draw line
            matrix[temp > 0] = value
            modified = True
        else:
            print("overlap happens, no line drawn for this group {}, {}, overalapping is: {}. ".format( start, end, np.sum(temp & ~ unoccpied_region)))
    return modified

def find_nearest_zero(*,matrix, start, mask, lines, force=False):
    '''Find the nearest zero point from start point in matrix
    Parameters:
        matrix: where to find the nearest zero
        start: start point
        mask: mask to avoid some regions
        lines: place to draw the line at the same time
        force: if True, ignore the overlapping
    Return:
        nearest_zero_point, direction
    '''
    # Directions to move in the matrix, representing all possible neighbors in 3D
    directions = [(1, 0, 0), (-1, 0, 0), (0, 1, 0), (0, -1, 0), (0, 0, 1), (0, 0, -1)]
    
    # Initialize a queue for BFS and add the starting point
    queue = deque([start])
    
    # Keep track of visited points to avoid loops
    visited = set()
    visited.add(tuple(start))
    
    # Matrix dimensions
    x_max, y_max, z_max = matrix.shape
    
    while queue:
        x, y, z = queue.popleft()
        
        # Check if the current location has the target value
        if matrix[x, y, z] != 0 and mask[x, y, z] == 0:
            # if Bresenham_Line_3d(lines,start,(x,y,z),1,False) == True:
            #     return (x, y, z), 1
            # elif Bresenham_Line_3d(lines,(x,y,z),start, 1,False) == True:
            #     return (x, y, z), -1
            if Diagonal_Line_3d(matrix=lines, start=start, end=(x,y,z), value=1, change=False, fitcytoplasmaCrop=fitcytoplasmaCrop, force=force,max_overlap=4) == True:
                return (x, y, z), 1
        
        # Explore all neighbors
        for dx, dy, dz in directions:
            nx, ny, nz = x + dx, y + dy, z + dz
            
            # Check if the neighbor is within bounds and not visited
            if 0 <= nx < x_max and 0 <= ny < y_max and 0 <= nz < z_max and (nx, ny, nz) not in visited:
                visited.add((nx, ny, nz))
                queue.append((nx, ny, nz))
    
    # Return None if no neighbor with value 0 is found
    return None,None

This code is for junctions between cecER and nuclear envelope, not important, our mRNA and protein will go through nuclear pores, not these junctions.

In [15]:
# # need a math conversion 
# necs_density = 0.05 # 0.05 junction/ micron^2, per NE surface area 
# necs_prob = necs_density * 10**(-6) * lattice_spacing**2 
# print("necs_prob:",necs_prob)
# NucEnvContactSite = np.zeros_like(cutcecERFullReg)
# NucEnvContactSite[cutcecERFullReg] = np.random.choice([0,1],size=np.sum(cutcecERFullReg),p=[1-necs_prob,necs_prob])
# # now connect the cecER and pmER
# necs = np.argwhere(NucEnvContactSite)


# neighbor,direction = find_nearest_zero(fitNucEnvCropRegion&~fitNucPoresCropRegion, cyto_sites_coord[node], , lines)

Add tunnels between cecER and pmER

https://bionumbers.hms.harvard.edu/bionumber.aspx?id=111388&ver=0&trm=endoplasmic+reticulum+yeast&org=

Shibata Y, Voeltz GK, Rapoport TA. Rough sheets and smooth tubules. Cell. 2006 Aug 11 126(3):435-9. p.435 left column bottom paragraphPubMed ID16901774

diameter of the tunnel is 60-100 nm


In [ ]:
# create the empty region for cuttubER
cuttubERFullReg = np.zeros_like(cutcecERFullReg)
# randomly generate the diameter form 60-100 nm
# tunnel_diameter = int(np.random.randint(60,101)/lattice_spacing)
tunnel_diameter = 3 # force to be 3 now 3 * 28 = 84 nm

# total_length = volume cubes of tubER / tunnel_dia
total_length = int(voxel_tubER / (tunnel_diameter)**2 * np.random.uniform(0.3, 0.5))
print("tunnel_diameter:",tunnel_diameter,"total_length:",total_length)

In [17]:
from collections import deque
import numpy as np

def bfs_3d_connect(matrix, start, end_points, thickness=1, max_length=float('inf')):
    """
    Perform 3D BFS to connect start point to a set of end points, then thicken the path.
    
    Args:
    matrix (np.array): 3D numpy array representing the space.
    start (tuple): Starting coordinates (x, y, z).
    end_points (set or list): Set or list of tuples representing end coordinates (x, y, z).
    thickness (int): Thickness of the path (default: 1).
    max_length (int): Maximum allowed path length (default: infinity).
    
    Returns:
    tuple: (thickened_path, length) where thickened_path is a list of coordinates and length is the original path length,
           or (None, None) if no path found within max_length.
    """
    start = tuple(start)
    end_points = set(map(tuple, end_points))
    
    queue = deque([(start, [start], 0)])
    visited = set([start])
    
    directions = [
        (1, 0, 0), (-1, 0, 0),
        (0, 1, 0), (0, -1, 0),
        (0, 0, 1), (0, 0, -1)
    ]
    
    def thicken_path(path, matrix, thickness):
        thickened_path = set()
        for i in range(len(path) - 1):
            current = path[i]
            next_point = path[i+1]
            direction = tuple(np.subtract(next_point, current))
            
            # Determine perpendicular directions
            if direction[0] != 0:  # x-dominant
                perp1, perp2 = (0, 1, 0), (0, 0, 1)
            elif direction[1] != 0:  # y-dominant
                perp1, perp2 = (1, 0, 0), (0, 0, 1)
            else:  # z-dominant
                perp1, perp2 = (1, 0, 0), (0, 1, 0)
            
            for x in range(-thickness//2 + 1, thickness//2 + 1):
                for y in range(-thickness//2 + 1, thickness//2 + 1):
                    offset = tuple(np.add(np.multiply(perp1, x), np.multiply(perp2, y)))
                    thickened_point = tuple(np.add(current, offset))
                    if all(0 <= thickened_point[i] < matrix.shape[i] for i in range(3)):
                        # Only add the point if it corresponds to a non-zero value in the matrix
                        if matrix[thickened_point] != 0:
                            thickened_path.add(thickened_point)
        
        return list(thickened_path)
    
    while queue:
        current, path, length = queue.popleft()
        
        if current in end_points:
            return thicken_path(path, matrix, thickness), length
        
        if length >= max_length:
            continue
        
        for direction in directions:
            next_point = tuple(np.add(current, direction))
            
            if (all(0 <= next_point[i] < matrix.shape[i] for i in range(3)) and 
                matrix[next_point] != 0 and 
                next_point not in visited):
                
                visited.add(next_point)
                new_path = path + [next_point]
                new_length = length + 1
                queue.append((next_point, new_path, new_length))
    
    return None, None  # No path found within max_length

# Example usage:
# matrix = np.ones((10, 10, 10))  # Create a 10x10x10 matrix filled with ones
# start = (0, 0, 0)
# end_points = [(9, 9, 9)]
# thickness = 3

# thickened_path, length = bfs_3d_connect(matrix, start, end_points, thickness=thickness, max_length=100)
# if thickened_path:
#     print(f"Path found with original length: {length}")
#     print(f"Number of points in thickened path: {len(thickened_path)}")
# else:
#     print("No path found within the specified maximum length")

In [ ]:
curr_length = 0
total_tunnels = 0
cuttubERFullReg = np.zeros_like(cutcecERFullReg)
# test case
# connect all regions 
num_unique_regions_pmER = len(np.unique(cutpmERFullReg)) - 1
num_unique_regions_cecER = len(np.unique(cutcecERFullReg)) - 1
connected_pmregions = []
connected_cecERregions = []
tried_pmregions = np.zeros(num_unique_regions_pmER + 1)
tried_cecregions = np.zeros(num_unique_regions_cecER + 1)
# create a limited trial before it gets stuck in the loop
trial = 0
repeated_trial = 0
MAX_TRIAL = 10
while curr_length < total_length:
    
    connected = False
    # Randomly find a cecER region that is not connected yet
    cecER_index = np.random.randint(1,num_unique_regions_cecER+1)
    tried_cecregions[cecER_index] += 1
    # if the cecER region is already connected and still exist non connected cecER regions
    # find another cecER region
    while cecER_index in connected_cecERregions and len(connected_cecERregions) < num_unique_regions_cecER:
        cecER_index = np.random.randint(1,num_unique_regions_cecER+1)
        tried_cecregions[cecER_index] += 1
        # if tried 10 times on the same cec Region still can't find a connection, give up
        if tried_cecregions[cecER_index] > int(MAX_TRIAL/2) and cecER_index not in connected_cecERregions:
            connected_cecERregions.append(cecER_index)
    
    valid_coords = np.argwhere(cutcecERFullReg == cecER_index)
    
    # Randomly choose one of these indices
    random_index = np.random.randint(0, len(valid_coords))
    start_pt = np.array(valid_coords[random_index])
    
    end_pt, IF_found = find_nearest_zero(matrix=cutpmERFullReg, start=start_pt, mask=np.zeros_like(cutpmERFullReg), lines=fitcytoplasmaCrop, force=True)
    if cutpmERFullReg[end_pt] == 0:
        print("end_pt is outside the pmER region, force stop")
        break
    if trial > MAX_TRIAL:
        print("trial > 10, force find random end point")
        repeated_trial += 1
        available_pmER_ids = set(range(1, num_unique_regions_pmER + 1)) - set(connected_pmregions)
        if available_pmER_ids:
            rand_pmER_id = np.random.choice(list(available_pmER_ids))
            connected_pmregions.append(rand_pmER_id)
        else:
            rand_pmER_id = np.random.randint(1, num_unique_regions_pmER + 1)
        
        valid_end_pts = np.argwhere(cutpmERFullReg == rand_pmER_id)
        end_pt = tuple(valid_end_pts[np.random.randint(0, len(valid_end_pts))])
   
    if end_pt is not None:
        # keep track of the pm ER regions visited times
        tried_pmregions[cutpmERFullReg[end_pt]] += 1
        
        # make sure at least all the pmER regions are connected to the cecER
        if cutpmERFullReg[end_pt] not in connected_pmregions or len(connected_pmregions) == num_unique_regions_pmER or trial > MAX_TRIAL:
           
            
            # connected, tunnel_length = Bresenham_Line_3d(matrix=cuttubERFullReg, start=start_pt, end=np.array(end_pt), value=1, thickness=tunnel_diameter, fitcytoplasmaCrop=fitcytoplasmaCrop, force=False, max_overlap=20)
            # use BFS instead of Bresenham_Line_3d
            
            avail_region = fitcytoplasmaCrop | cutcecERFullReg| cutcecERFullReg
            # avail_region &= ~RegionBuilder.dilate(fitMitoCropReg ,int(tunnel_diameter/2))
            # avail_region &= ~RegionBuilder.dilate(fitVacuoleCropReg, int(tunnel_diameter/2))
            connected, tunnel_length = bfs_3d_connect(matrix=avail_region, start=start_pt, end_points=np.argwhere(cutpmERFullReg == cutpmERFullReg[end_pt]), thickness=tunnel_diameter, max_length=total_length)
            
            if connected:
                #dont include the start and end point in the cuttubERFullReg
                for point in connected[1:-1]:
                    cuttubERFullReg[point] = 1
                trial = 0
                print("start_pt:",start_pt, "cecER region index:",cecER_index,  "tried_cecregions times:",tried_cecregions[cecER_index], "trial:",trial,)
                print("end_pt:",end_pt, "end point region:",cutpmERFullReg[end_pt], " pm ER tried times:",tried_pmregions[cutpmERFullReg[end_pt]])
                total_tunnels += 1
                curr_length += tunnel_length
                if cutpmERFullReg[end_pt] not in connected_pmregions:
                    
                    connected_pmregions.append(cutpmERFullReg[end_pt])
                if cecER_index not in connected_cecERregions:
                    connected_cecERregions.append(cecER_index)
                print("curr_length:",curr_length)
        else:
            trial += 1
        # if try to connect to the same region more than 10 times, give up 
        if tried_pmregions[cutpmERFullReg[end_pt]] > int(MAX_TRIAL/2) and cutpmERFullReg[end_pt] not in connected_pmregions:
            connected_pmregions.append(cutpmERFullReg[end_pt])
        # if successful connect to the pmER region, - 1 the connected_pmregions
        if connected:
            if tried_pmregions[cutpmERFullReg[end_pt]] > 0:
                tried_pmregions[cutpmERFullReg[end_pt]] -=1
            if tried_cecregions[cecER_index] > 0:
                tried_cecregions[cecER_index] -=1
    if repeated_trial > 4:
        print("repeated_trial > 4, break the loop")
        break
    if connected:
        # print out the connected pmER and cecER regions
        print("connected_pmregions:",connected_pmregions)
        print("connected_cecERregions:",connected_cecERregions)
        print("----------------------------------")
print("total_tunnels:",total_tunnels)


remove tub ER from cytoplasm Region

In [ ]:
cuttubERFullReg = (cuttubERFullReg > 0).astype(bool)
cuttubERFullReg &= ~ (cutcecERFullReg_binary | cutpmERFullReg_binary)
fitcytoplasmaCrop &= ~cuttubERFullReg

In [ ]:
RegionBuilder.showBinaryLattices(dict(cuttubERFullReg=cuttubERFullReg,
                                      pmER=cutpmERFullReg, 
                                      cecER=cutcecERFullReg,
                                      cytoplasm = fitcytoplasmaCrop))

In [ ]:

fitcytoplasmaCrop &= ~( fitNucEnvCropRegion | fitNucPoresCropRegion |
                       cutpmERFullReg_binary | cutcecERFullReg_binary)

In [ ]:
# check the overlap with each other, if exist, remove
if 'Tubular ER' not in region_names:
    regions_to_check.extend([cuttubERFullReg])
    region_names.extend(['Tubular ER'])

overlaps = check_overlaps(regions_to_check, region_names)

if overlaps:
    print("Overlaps detected:")
    for overlap in overlaps:
        print(f"Overlap between {overlap[0]} and {overlap[1]}: {overlap[2]} voxels")
else:
    print("No overlaps detected. Proceeding with lattice assignment.")
    
    
unoccupied_reg = fully_occupied_check(regions_to_check)

In [ ]:
# Save the ER region data as .npy files to prevent data loss in case of kernel crash
import numpy as np

# Define the file names
pm_er_file = './er_tunnels/pmER_geometry.npy'
cec_er_file = './er_tunnels/cecER_geometry.npy'
tub_er_file = './er_tunnels/tubER_geometry.npy'

# Save the ER region data
np.save(pm_er_file, cutpmERFullReg)
np.save(cec_er_file, cutcecERFullReg)
np.save(tub_er_file, cuttubERFullReg)

print(f"ER region data saved as {pm_er_file}, {cec_er_file}, and {tub_er_file}")


these two regions are separate:

---